This Jupyter notebook is made available as part of our work

N. Yapa, K. Fossez, S. Koenig, _Eigenvector continuation for emulating and
extrapolating two-body resonances_.

While it has been thoroughly tested, no guarantee is given that the notebook and
the supporting pacakge "twobodyEC.py" are fee of errors or that they will be fit
for a particular desired purpose.

Permission is granted to use the notebook and package as basis for further
scientific work under the conditions that:

1. An attribution to the original code is made in the published result,
   preferably by citing our work as mentioned above.
    
2. The derived or extended program code is made available together with
   the new published work, under the same or compatible conditions as stated
   here.

In [ ]:
from twobodyEC import *
import random

phi = pi/6
cutoff = 8.0
N = 256
gauss_mesh = True
datasets = 128
seed = 12345

random.seed(seed)
np.random.seed(seed)

## S-wave system

In [ ]:
l = 0

def V0(c, q, qp):
    return c*(-5*gaussian_s(1/3, q, qp)+2*gaussian_s(1/10, q, qp))

### Resonance-to-resonance

In [ ]:
# Figure 5

EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V0, l, gauss_mesh)
EC.train((0.78, 0.45), 5, line_plot_points=20)
EC.finish_training()
EC.extrapolate(np.linspace(0.38, 0.22, 5))
EC.plot((0, 0.25), (-0.225, 0), fig_size=(5, 5))

### Bound-state-to-resonance

In [ ]:
training_region = (1.3, 0.9)
training_points = 5
xlim = (-0.6, 0.35)
ylim = (-0.35, 0.1)
fig_size = (7.5, 7.5)

In [ ]:
# Figure 6

EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V0, l, gauss_mesh)
EC.train(training_region, training_points)
EC.finish_training(augment=None)
EC.extrapolate(np.linspace(0.77, 0.22, 6))
EC.plot(xlim, ylim, fig_size=fig_size)

In [ ]:
# Figure 7

EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V0, l, gauss_mesh)
EC.train(training_region, training_points)
EC.finish_training(augment='conj')
EC.extrapolate(np.linspace(0.77, 0.22, 6))
EC.plot(xlim, ylim, fig_size=fig_size)

In [ ]:
# Figure 9

EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V0, l, gauss_mesh)
EC.train(training_region, training_points)
EC.finish_training(augment='RH')
EC.extrapolate(np.linspace(0.77, 0.22, 6))
EC.plot(xlim, ylim, fig_size=fig_size)

### Convergece with increasing number of RH functions

In [ ]:
# Figure 10

for no_points in range(1, training_points):
    EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V0, l, gauss_mesh)
    EC.train(training_region, training_points)

    # add RH vectors for a random sample of points
    for model in EC.models:
        for E in random.sample(model.training_E, k=no_points):
            k = sqrt(E)*exp(-2j*model.rotation_angle)
            new_wf = RH_function(k, model.s.q, model.l)
            model.training_vecs.append(new_wf)

    EC.finish_training()
    EC.extrapolate(np.linspace(0.77, 0.22, 6))
    EC.plot(xlim, ylim, fig_size=fig_size)

## P-wave system

In [ ]:
l = 1

def V1(c, q, qp):
    return -c*gaussian_p(1/4, q, qp)

In [ ]:
# Figure 8

EC = two_body_EC_ensemble(datasets, N, cutoff, phi, V1, l, gauss_mesh)
EC.train((4, 3.1), 5)
EC.finish_training(augment='conj')
EC.extrapolate(np.linspace(1.8, 2.9, 5))
EC.plot((-0.3, 0.16), (-0.2, 0.05), fig_size=(6, 4))